In [14]:
import numpy as np
from math import log
import sys
from sympy import Poly, symbols, GF, invert

def factor_int(n):
    """
    Funkcja zwraca słownik z rozkładem na czynniki pierwsze liczby n.
    Kluczami słownika są czynniki pierwsze liczby n, a wartościami są ich
    krotności (czyli potęgi, do których każdy czynnik pierwszy występuje w rozkładzie).
    """
    factors_ = {}
    d = 2
    while n > 1:
        while n % d == 0:
            if d in factors_:
                factors_[d] += 1
            else:
                factors_[d] = 1
            n //= d
        d += 1
        if d*d > n:
            if n > 1:
                if n in factors_:
                    factors_[n] += 1
                else:
                    factors_[n] = 1
            break
    return factors_


def checkPrime(P):
    """
    Funkcja sprawdza, czy podana liczba całkowita P jest liczbą pierwszą, jeśli tak, zwraca True
    w przeciwnym razie zwraca False.
    """
    if P <= 1:
        return False
    elif P == 2 or P == 3:
        return True
    else:
        for i in range(4, P // 2):
            if P % i == 0:
                return False
    return True


def poly_inv(poly_in, poly_I, poly_mod):
    """
    Funkcja znajduje odwrotność wielomianu poly_in w ciele Galois GF(poly_mod)
    tj. odwrotność w Z/poly_mod[X]/poly_I

    Zwraca albo pustą tablicę, jeśli nie można znaleźć odwrotności, albo odwrotność
    wielomianu poly_in jako tablicę współczynników.
    """
    x = symbols('x')
    Ppoly_I = Poly(poly_I, x)
    Npoly_I = len(Ppoly_I.all_coeffs())
    if checkPrime(poly_mod):
        try:
            inv = invert(Poly(poly_in, x).as_expr(), Ppoly_I.as_expr(), domain=GF(poly_mod, symmetric=False))
        except:
            return np.array([])
    elif log(poly_mod, 2).is_integer():
        try:
            inv = invert(Poly(poly_in, x).as_expr(), Ppoly_I.as_expr(), domain=GF(2, symmetric=False))
            ex = int(log(poly_mod, 2))
            for a in range(1, ex):
                inv = ((2 * Poly(inv, x) - Poly(poly_in, x) * Poly(inv, x) ** 2) % Ppoly_I).trunc(poly_mod)
            inv = Poly(inv, domain=GF(poly_mod, symmetric=False))
        except:
            return np.array([])
    else:
        return np.array([])

    tmpCheck = np.array(Poly((Poly(inv, x) * Poly(poly_in, x)) % Ppoly_I,
                             domain=GF(poly_mod, symmetric=False)).all_coeffs(), dtype=int)
    if len(tmpCheck) > 1 or tmpCheck[0] != 1:
        sys.exit("BŁĄD: Błąd w obliczeniu odwrotności wielomianu")

    return padArr(np.array(Poly(inv, x).all_coeffs(), dtype=int), Npoly_I - 1)


def padArr(A_in, A_out_size):
    """
    Funkcja wraca tablicę numpy o rozmiarze A_out_size z wiodącymi zerami
    """
    return np.pad(A_in, (A_out_size - len(A_in), 0), constant_values=0)


def genRand10(L, P, M):
    """
    Funkcja generuje tablicę numpy o długości L z P jedynkami, M minus jedynkami, a pozostałe elementy to zera.
    Jest to używane do generowania tablic f, p i r dla szyfrowania NTRU.

    L : Liczba całkowita, długość pożądanej tablicy wyjściowej.
    P : Liczba całkowita, liczba `pozytywnych` tj. +1 w tablicy.
    M : Liczba całkowita, liczba `negatywnych` tj. -1 w tablicy.
    """

    if P + M > L:
        sys.exit("BŁĄD: Żądanie P + M > L.")

    R = np.zeros((L,), dtype=int)

    for i in range(L):
        if i < P:
            R[i] = 1
        elif i < P + M:
            R[i] = -1
        else:
            break

    np.random.shuffle(R)
    return R


def arr2str(ar):
    """
    Funkcja konwertuje tablicę numpy na ciąg znaków zawierający tylko elementy tablicy.
    """
    st = np.array_str(ar)
    st = st.replace("[", "", 1)
    st = st.replace("]", "", 1)
    st = st.replace("\n", "")
    st = st.replace("     ", " ")
    st = st.replace("    ", " ")
    st = st.replace("   ", " ")
    st = st.replace("  ", " ")
    return st

def str2bit(st):
    """
    Funkcja konwertuje wejściowy ciąg znaków st na binarną reprezentację ciągu, z każdym
    bitem jako elementem całkowitej tablicy numpy.

    UWAGA: Początkowe "0b" jest usuwane z tablicy wyjściowej.
    """
    return np.array(list(bin(int.from_bytes(str(st).encode(), "big")))[2:], dtype=int)


def bit2str(bi):
    """
    Funkcja konwertuje tablicę bitów na ciąg znaków opisany przez te bity.
    """

    S = arr2str(bi)
    S = S.replace(" ", "")

    charOut = ""
    for i in range(len(S) // 8):
        if i == 0:
            charb = S[len(S) - 8:]
        else:
            charb = S[-(i + 1) * 8:-i * 8]
        charb = int(charb, 2)
        charOut = charb.to_bytes((charb.bit_length() + 7) // 8, "big").decode("utf-8", errors="ignore") + charOut
    return charOut

In [15]:
class NTRUencrypt:
    """
    Klasa do szyfrowania danych na podstawie znanego klucza publicznego.
    """

    def __init__(self, N=503, p=3, q=256, d=18):
        """
        Inicjalizacja z domyślnymi parametrami N, p i q.
        """
        self.N = N  # Publiczne N
        self.p = p  # Publiczne p
        self.q = q  # Publiczne q

        self.dr = d  # Liczba jedynek w r (do szyfrowania)

        self.g = np.zeros((self.N,), dtype=int)  # Prywatny wielomian g
        self.h = np.zeros((self.N,), dtype=int)  # Publiczny wielomian klucza (mod q)
        self.r = np.zeros((self.N,), dtype=int)  # Losowa wartość
        self.genr()
        self.m = np.zeros((self.N,), dtype=int)  # Tablica wiadomości
        self.e = np.zeros((self.N,), dtype=int)  # Zaszyfrowana wiadomość

        self.I = np.zeros((self.N + 1,), dtype=int)
        self.I[self.N] = -1
        self.I[0] = 1

        self.readKey = False  # Klucz publiczny nie został jeszcze odczytany

        self.Me = None  # Zaszyfrowana wiadomość jako string


    def readPub(self, filename="key.pub"):
        """
        Odczytanie pliku klucza publicznego, wygenerowanie nowej wartości r na podstawie nowego N
        """
        with open(filename, "r") as f:
            self.p = int(f.readline().split(" ")[-1])
            self.q = int(f.readline().split(" ")[-1])
            self.N = int(f.readline().split(" ")[-1])
            self.dr = int(f.readline().split(" ")[-1])
            self.h = np.array(f.readline().split(" ")[3:-1], dtype=int)
        self.I = np.zeros((self.N + 1,), dtype=int)
        self.I[self.N] = -1
        self.I[0] = 1
        self.genr()
        self.readKey = True

    def genr(self):
        """
        Wygenerowanie losowego wielomianu maskującego r, z wartościami mod q
        """
        self.r = genRand10(self.N, self.dr, self.dr)

    def setM(self, M):
        """
        Ustawienie wiadomości klasy M po przeprowadzeniu kontroli błędów.
        Przed wywołaniem tej funkcji wartości klucza publicznego muszą być ustawione (tj. odczytane)
        UWAGA: Wiadomość M musi być tablicą opisującą współczynniki wielomianu, gdzie
               wielomian musi być stopnia < N.
        UWAGA: Współczynniki muszą być w zakresie [-p/2, p/2].
        UWAGA: Tablica wiadomości musi być tablicą całkowitą.
        """
        if not self.readKey:
            sys.exit("BŁĄD: Klucz publiczny nie został odczytany przed ustawieniem wiadomości")
        if len(M) > self.N:
            sys.exit("BŁĄD: Długość wiadomości jest dłuższa niż stopień ideału pierścienia wielomianów")
        for i in range(len(M)):
            if M[i] < -self.p / 2 or M[i] > self.p / 2:
                sys.exit("BŁĄD: Elementy wiadomości muszą być w zakresie [-p/2, p/2]")

        self.m = padArr(M, self.N)

    def encrypt(self, m=None):
        """
        Zaszyfrowanie wiadomości m do tablicy e
        UWAGA: Wiadomość m musi być ustawiona przed wywołaniem tej procedury
        """
        if not self.readKey:
            sys.exit("Błąd: Nie odczytano pliku klucza publicznego, więc nie można zaszyfrować")
        if m is not None:
            if len(m) > self.N:
                sys.exit("\n\nBŁĄD: Wiadomość wielomianowa stopnia >= N")
            self.m = m
        x = symbols('x')

        self.e = np.array(((((Poly(self.r, x) * Poly(self.h, x)).trunc(self.q))
                            + Poly(self.m, x)) % Poly(self.I, x)).trunc(self.q).all_coeffs(), dtype=int)
        self.e = padArr(self.e, self.N)

    def encryptString(self, M):
        if not self.readKey:
            sys.exit("Błąd: Nie odczytano pliku klucza publicznego, więc nie można zaszyfrować")

        bM = str2bit(M)
        bM = padArr(bM, len(bM) - np.mod(len(bM), self.N) + self.N)

        self.Me = ""

        for E in range(len(bM) // self.N):
            self.genr()
            self.setM(bM[E * self.N:(E + 1) * self.N])
            self.encrypt()
            self.Me = self.Me + arr2str(self.e) + " "

In [16]:
from math import gcd

class NTRUdecrypt:
    """
    Klasa do odszyfrowywania danych za pomocą metody NTRU.

    Ta klasa może również generować klucz prywatny używany do odszyfrowywania (który można zapisać do
    zewnętrznego pliku) oraz klucz publiczny używany do szyfrowania (który również można zapisać do
    zewnętrznego pliku).
    """

    def __init__(self, N=503, p=3, q=256, df=61, dg=20, d=18):
        """
        Inicjalizacja parametrów klucza i tablic wielomianów

        N  : Liczba całkowita, rząd pierścienia wielomianów
        p  : Liczba całkowita, moduł odwrotności wielomianu f dla fp
        q  : Liczba całkowita, moduł odwrotności wielomianu f dla fq
        df : Liczba całkowita, liczba współczynników równych 1 w wielomianie f
        dg : Liczba całkowita, liczba współczynników równych 1 w wielomianie g
        gr : Liczba całkowita, liczba współczynników równych 1 w losowym wielomianie (używanym w szyfrowaniu)
        """

        self.N = N  # Publiczne N
        self.p = p  # Publiczne p
        self.q = q  # Publiczne q

        self.df = df  # Liczba jedynek w f
        self.dg = dg  # Liczba jedynek w g
        self.dr = d  # Liczba jedynek w r (dla szyfrowania)

        self.f = np.zeros((self.N,), dtype=int)  # Prywatny wielomian f
        self.fp = np.zeros((self.N,), dtype=int)  # Odwrotność f mod p
        self.fq = np.zeros((self.N,), dtype=int)  # Odwrotność f mod q
        self.g = np.zeros((self.N,), dtype=int)  # Prywatny wielomian g
        self.h = np.zeros((self.N,), dtype=int)  # Publiczny wielomian klucza (mod q)

        self.I = np.zeros((self.N + 1,), dtype=int)
        self.I[self.N] = -1
        self.I[0] = 1

        self.M = None

    def setNpq(self, N=None, p=None, q=None, df=None, dg=None, d=None):
        """
        Ustawia wartości N, p i q oraz sprawdza warunki:
          - N musi być liczbą pierwszą
          - q musi być większe niż p
          - p i q muszą być względnie pierwsze
        """
        if N is not None:
            if not checkPrime(N):
                sys.exit("\n\nERROR: Wartość N nie jest liczbą pierwszą\n\n")
            else:
                # Sprawdzenie, czy liczba 1 w df, dg i dr jest poprawna w stosunku do N
                if df is None:
                    if 2 * self.df > N:
                        sys.exit("\n\nERROR: Wartość N jest za mała w porównaniu do domyślnego df " + str(self.df) + "\n\n")
                if dg is None:
                    if 2 * self.dg > N:
                        sys.exit("\n\nERROR: Wartość N jest za mała w porównaniu do domyślnego dg " + str(self.dg) + "\n\n")
                if d is None:
                    if 2 * self.dr > N:
                        sys.exit("\n\nERROR: Wartość N jest za mała w porównaniu do domyślnego dr " + str(self.dr) + "\n\n")
                # W przeciwnym razie, ustawienie N i inicjalizacja tablicy wielomianów
                self.N = N
                self.f = np.zeros((self.N,), dtype=int)
                self.fp = np.zeros((self.N,), dtype=int)
                self.fq = np.zeros((self.N,), dtype=int)
                self.g = np.zeros((self.N,), dtype=int)
                self.h = np.zeros((self.N,), dtype=int)
                self.I = np.zeros((self.N + 1,), dtype=int)
                self.I[self.N] = -1
                self.I[0] = 1

        if (p is None and q is not None) or (p is not None and q is None):
            sys.exit("\n\nError: Można ustawiać p i q tylko razem, nie indywidualnie")
        elif (p is not None) and (q is not None):
            if (8 * p) > q:
                sys.exit("\n\nERROR: Wymagane jest 8p <= q\n\n")
            else:
                if gcd(p, q) != 1:
                    sys.exit("\n\nERROR: Wartości p i q nie są względnie pierwsze\n\n")
                else:
                    self.p = p
                    self.q = q

        if df is not None:
            if 2 * df > self.N:
                sys.exit("\n\nERROR: Wartość df musi być taka, że 2*df>N\n\n")
            else:
                self.df = df

        if dg is not None:
            if 2 * dg > self.N:
                sys.exit("\n\nERROR: Wartość dg musi być taka, że 2*dg>N\n\n")
            else:
                self.dg = dg

        if d is not None:
            if 2 * d > self.N:
                sys.exit("\n\nERROR: Wartość dr musi być taka, że 2*dr>N\n\n")
            else:
                self.dr = d

    def invf(self):
        """
        Odwraca wielomian f względem wartości p i q (wartości klasy).
        Zwraca True, jeśli odwrotności względem p i q istnieją (po ustawieniu self.fp i self.fq)
        Zwraca False, jeśli odwrotność względem p lub q nie istnieje
        """
        fp_tmp = poly_inv(self.f, self.I, self.p)
        fq_tmp = poly_inv(self.f, self.I, self.q)
        if len(fp_tmp) > 0 and len(fq_tmp) > 0:
            self.fp = np.array(fp_tmp)
            self.fq = np.array(fq_tmp)
            # Upewnienie się, że tablice mają wiodące zera
            if len(self.fp) < self.N:
                self.fp = np.concatenate([np.zeros(self.N - len(self.fp), dtype=int), self.fp])
            if len(self.fq) < self.N:
                self.fq = np.concatenate([np.zeros(self.N - len(self.fq), dtype=int), self.fq])
            return True
        else:
            return False

    def genfg(self):
        """
        Losowo generuje f i g dla klucza prywatnego oraz ich odwrotności
        """
        maxTries = 100
        self.g = genRand10(self.N, self.dg, self.dg)
        # Wygenerowanie f z odwrotnościami mod p i mod q
        for i in range(maxTries):
            self.f = genRand10(self.N, self.df, self.df - 1)
            invStat = self.invf()
            if invStat:
                break
            elif i == maxTries - 1:
                sys.exit("Cannot generate required inverses of f")

    def genh(self):
        """
        Generuje klucz publiczny na podstawie wartości klasy (które muszą być wcześniej wygenerowane)
        """
        x = symbols('x')
        self.h = Poly((Poly(self.p * self.fq, x).trunc(self.q) * Poly(self.g, x)).trunc(self.q) \
                      % Poly(self.I, x)).all_coeffs()

    def writePub(self, filename="key"):
        """
        Zapisuje plik klucza publicznego
        """
        pubHead = "p ::: " + str(self.p) + "\nq ::: " + str(self.q) + "\nN ::: " + str(self.N) \
                  + "\nd ::: " + str(self.dr) + "\nh :::"
        np.savetxt(filename + ".pub", self.h, newline=" ", header=pubHead, fmt="%s")

    def readPub(self, filename="key.pub"):
        """
        Odczytuje plik klucza publicznego
        """
        with open(filename, "r") as f:
            self.p = int(f.readline().split(" ")[-1])
            self.q = int(f.readline().split(" ")[-1])
            self.N = int(f.readline().split(" ")[-1])
            self.dr = int(f.readline().split(" ")[-1])
            self.h = np.array(f.readline().split(" ")[3:-1], dtype=int)
        self.I = np.zeros((self.N + 1,), dtype=int)
        self.I[self.N] = -1
        self.I[0] = 1

    def writePriv(self, filename="key"):
        """
        Zapisuje plik klucza prywatnego
        """
        privHead = "p ::: " + str(self.p) + "\nq ::: " + str(self.q) + "\nN ::: " \
                   + str(self.N) + "\ndf ::: " + str(self.df) + "\ndg ::: " + str(self.dg) \
                   + "\nd ::: " + str(self.dr) + "\nf/fp/fq/g :::"
        np.savetxt(filename + ".priv", (self.f, self.fp, self.fq, self.g), header=privHead, newline="\n", fmt="%s")

    def readPriv(self, filename="key.priv"):
        """
        Odczytuje plik klucza prywatnego
        """
        with open(filename, "r") as f:
            self.p = int(f.readline().split(" ")[-1])
            self.q = int(f.readline().split(" ")[-1])
            self.N = int(f.readline().split(" ")[-1])
            self.df = int(f.readline().split(" ")[-1])
            self.dg = int(f.readline().split(" ")[-1])
            self.dr = int(f.readline().split(" ")[-1])

            lines = f.readlines()
            data_lines = [line for line in lines if not line.strip().startswith("#")]

            self.f = np.array(data_lines[0].split(), dtype=int)
            self.fp = np.array(data_lines[1].split(), dtype=int)
            self.fq = np.array(data_lines[2].split(), dtype=int)
            self.g = np.array(data_lines[3].split(), dtype=int)

        self.I = np.zeros((self.N + 1,), dtype=int)
        self.I[self.N] = -1
        self.I[0] = 1

    def genPubPriv(self, keyfileName="key"):
        """
        Generuje klucze publiczne i prywatne z wartości klasy N, p i q.
        Zapisuje również pliki wyjściowe dla kluczy publicznych i prywatnych.
        """
        self.genfg()
        self.genh()
        self.writePub(keyfileName)
        self.writePriv(keyfileName)

    def decrypt(self, e):
        """
        Odszyfrowuje wiadomość podaną jako tablicę e do odszyfrowanej wiadomości m i zwraca ją.
        """
        # Zaszyfrowana wiadomość e musi mieć stopień < N
        if len(e) > self.N:
            sys.exit("Zaszyfrowana wiadomość ma stopień > N")
        # Odszyfrowanie i zwrócenie jako tablicę numpy
        x = symbols('x')
        a = ((Poly(self.f, x) * Poly(e, x)) % Poly(self.I, x)).trunc(self.q)
        b = a.trunc(self.p)
        c = ((Poly(self.fp, x) * b) % Poly(self.I, x)).trunc(self.p)

        return np.array(c.all_coeffs(), dtype=int)

    def decryptString(self, E):
        """
        Odszyfrowuje wiadomość zakodowaną za pomocą odpowiedniego klucza publicznego z zaszyfrowanego ciągu do odszyfrowanego ciągu znaków.
        """

        Me = np.fromstring(E, dtype=int, sep=' ')
        if np.mod(len(Me), self.N) != 0:
            sys.exit("\n\nERROR : Wejściowy ciąg odszyfrowujący nie jest całkowitą wielokrotnością N\n\n")

        # Odszyfrowanie każdego bloku, dodając do ciągu wiadomości
        Marr = np.array([], dtype=int)
        for D in range(len(Me) // self.N):
            Marr = np.concatenate((Marr, padArr(self.decrypt(Me[D * self.N:(D + 1) * self.N]), self.N)))

        self.M = bit2str(Marr)

In [17]:
def encrypt(name: str, string: str):
    """
    name: nazwa pliku klucza
    string: wiadomość do zaszyfrowania jako ciąg znaków
    """
    E = NTRUencrypt()
    E.readPub(name + ".pub")
    E.encryptString(string)

    return E.Me


def decrypt(name: str, cipher: str):
    """
    name: nazwa pliku klucza
    cipher: zaszyfrowana wiadomość
    """
    D = NTRUdecrypt()
    D.readPriv(name + ".priv")
    to_decrypt = cipher
    D.decryptString(to_decrypt)

    return D.M


def generate_keys_ntru(name="key", mode="highest", skip_check=False, debug=False):
    if mode not in ["moderate", "high", "highest"]:
        raise ValueError("Wartość wejściowa musi być 'moderate', 'high' lub 'highest'")
    """
    name: nazwa pliku wyjściowego
    mode: moderate, high, highest
    """
    if debug:
        finished = False
        print("[i] Rozpoczynanie generowania...")
        for i in range(10):
            print(f"[i] Runda {i}/10 generowania klucza rozpoczęta")
            N1 = NTRUdecrypt()
            print("[i] Funkcja zainicjalizowana.")
            print("Wybór trybu:", mode)
            if mode == "moderate":
                N1.setNpq(N=107, p=3, q=64, df=15, dg=12, d=5)
            elif mode == "high":
                N1.setNpq(N=167, p=3, q=128, df=61, dg=20, d=18)
            elif mode == "highest":
                N1.setNpq(N=503, p=3, q=256, df=216, dg=72, d=55)

            print("[i] Generowanie kluczy")
            N1.genPubPriv(name)
            print("[i] Utworzono.")

            if skip_check:
                finished = True
                print("[-] Pomijanie sprawdzenia bezpieczeństwa")
                break

            print("[i] Pobieranie czynników:")
            factors = factor_int(N1.h[-1])
            print("[i] Czynniki:", factors)
            possible_keys = 2 ** N1.df * (N1.df + 1) ** 2 * 2 ** N1.dg * (N1.dg + 1) * 2 ** N1.dr * (N1.dr + 1)
            print("[i] Sprawdzanie, czy klucz jest wystarczająco długi.")
            if len(factors) == 0 and possible_keys > 2 ** 80:
                print("Bezpieczeństwo potwierdzone")
                finished = True
                break
            else:
                print("[-] Sprawdzenie bezpieczeństwa nie powiodło się. Próbowanie ponownie.")
        if not finished:
            print(
                "[-] Nie udało się wygenerować klucza, ponieważ nie można było zweryfikować bezpieczeństwa w 10 próbach.")
        print("[+] Gotowe.")
    else:
        finished = False
        for i in range(10):
            N1 = NTRUdecrypt()
            if mode == "moderate":
                N1.setNpq(N=107, p=3, q=2048, df=15, dg=12, d=5)
            elif mode == "high":
                N1.setNpq(N=167, p=3, q=2048, df=61, dg=20, d=18)
            elif mode == "highest":
                N1.setNpq(N=503, p=3, q=2048, df=216, dg=72, d=55)

            N1.generate_keys_with_rsa(name)

            if skip_check:
                finished = True
                break

            factors = factor_int(N1.h[-1])
            possible_keys = 2 ** N1.df * (N1.df + 1) ** 2 * 2 ** N1.dg * (N1.dg + 1) * 2 ** N1.dr * (N1.dr + 1)
            if len(factors) == 0 and possible_keys > 2 ** 80:  # zobacz 'security_check.py' dla więcej informacji
                finished = True
                break
            print("Niepowodzenie")
        if not finished:
            print("[-] Nie można było zweryfikować bezpieczeństwa kluczy.")


if __name__ == "__main__":
    # Generowanie kluczy
    generate_keys_ntru(name="example_key", mode="high", debug=True)

    # Szyfrowanie wiadomości
    encrypted_message = encrypt("example_key", "nie lubie kryptografii")
    print("Encrypted message:", encrypted_message)

    # Deszyfrowanie wiadomości
    decrypted_message = decrypt("example_key", encrypted_message)
    print("Decrypted message:", decrypted_message)

[i] Rozpoczynanie generowania...
[i] Runda 0/10 generowania klucza rozpoczęta
[i] Funkcja zainicjalizowana.
Wybór trybu: high
[i] Generowanie kluczy
[i] Utworzono.
[i] Pobieranie czynników:
[i] Czynniki: {}
[i] Sprawdzanie, czy klucz jest wystarczająco długi.
Bezpieczeństwo potwierdzone
[+] Gotowe.
Encrypted message:  39 42 -5 -47 37 -31 -46 -46 -9 15 8 57 -31 -3 31 -1 -53 38 -34 -48 -3 -59 15 52 -37 -31 -1 57 -36 29 16 -31 -20 37 9 17 0 -54 -36 41 -27 -52 15 57 1 -62 16 -18 -8 16 -39 5 -53 -28 48 33 9 -43 23 -55 -28 24 -50 -52 -53 47 -48 -31 -21 -31 -7 51 -41 32 56 -32 47 40 -27 -31 -52 8 -17 4 -30 2 -60 35 -38 21 29 -18 -63 17 41 -46 57 64 -27 54 -35 -36 4 53 -2 -61 -6 40 50 -45 -51 -55 16 -6 -16 -52 23 -26 47 2 -40 47 31 -3 52 -9 48 -40 -26 -40 -20 -3 2 -14 63 56 59 -56 -41 -18 18 -6 -55 0 -43 -4 0 31 42 63 0 61 61 49 64 -1 -56 17 -24 14 22 -31 -27 16 -5 9 -28 -21 -26 52 60 -24 -44 43 14 34 49 -26 -46 -39 -61 2 -21 -1 -22 15 -4 22 11 -34 9 -40 -1 -1 60 48 -13 43 18 -57 7 -29 19 -17 